# Irresoluteness Data for Appendix D

In [1]:
from pref_voting.profiles import *
from pref_voting.generate_profiles import *
from pref_voting.generate_weighted_majority_graphs import *
from pref_voting.voting_methods import *
from functools import partial
from multiprocess import Pool, cpu_count, current_process
import pandas as pd
import os

In [ ]:
def init_process():
    np.random.seed(os.getpid())


In [ ]:
def ws_data(vms, num_cands, num_voters, pm, t): 
    if not hasattr(ws_data, 'seed_set'):
        init_process()
        ws_data.seed_set = True
    prof = generate_profile(num_cands, num_voters, probmod = pm )
    winning_sets = {vm.name: vm(prof) for vm in vms}
    return {vm.name: len(winning_sets[vm.name]) if winning_sets[vm.name] is not None else None 
            for vm in vms}

In [ ]:
vms = [
    split_cycle,
    beat_path_Floyd_Warshall, 
    copeland, 
    getcha,
    uc_gill,
]

num_trials = 25_000

all_num_voters = [
    (4, 5),
    (10, 11), 
    (20, 21), 
    (50, 51), 
    (100, 101),
    (500, 501),
    (1000, 1001),
    (5000, 5001)
]

all_num_cands = [
    5,
    7, 
    10, 
    20, 
    30
]

prob_models = [
    "IC", 
    "IAC",
    "URN-R",
    "MALLOWS-RELPHI-R",
    "MALLOWS_2REF-RELPHI-R"
]
cpus = cpu_count()
print(f'CPUS: {cpus}')
num_cpus = 10

pool = Pool(num_cpus)

In [ ]:
%%time 

data_for_df = {
    "num_cands": list(), 
    "num_voters": list(),
    "pm": list(),
    "vm": list(),
    "num_profiles": list(),
    "perc_mult_winners": list(),
}

data_for_df2 = {
    "num_cands": list(), 
    "num_voters": list(),
    "pm": list(),
    "vm": list(),
    "size_mult_winners": list(),
}

for pm in prob_models:
    print(pm)
    for num_cands in all_num_cands:
        print(num_cands)
        for num_voters_even, num_voters_odd in all_num_voters:

            print(f"\t{num_voters_even}, {num_voters_odd}")

            get_data_even = partial(ws_data, 
                                    vms,
                                    num_cands,
                                    num_voters_even,
                                    pm)

            get_data_odd = partial(ws_data, 
                                   vms,
                                   num_cands,
                                   num_voters_odd,
                                   pm)

            data_even = pool.map(get_data_even, range(num_trials))
            data_odd = pool.map(get_data_odd, range(num_trials))

            data = data_even + data_odd

            num_profiles = len(data)
            for vm in vms: 
                data_for_df["num_cands"].append(num_cands)
                data_for_df["num_voters"].append(str((num_voters_even, num_voters_odd)))
                data_for_df["vm"].append(vm.name)
                data_for_df["pm"].append(pm)
                num_profiles = len([1 for d in data if d[vm.name] is not None])
                data_for_df["num_profiles"].append(num_profiles)
                data_for_df["perc_mult_winners"].append(len([1 for d in data if d[vm.name] > 1]) / num_profiles)

            for d in data: 
                for vm in vms:
                    if d[vm.name] > 1:
                        data_for_df2["num_cands"].append(num_cands)
                        data_for_df2["num_voters"].append(str((num_voters_even, num_voters_odd)))
                        data_for_df2["vm"].append(vm.name)
                        data_for_df2["pm"].append(pm)
                        data_for_df2["size_mult_winners"].append(d[vm.name])

        df = pd.DataFrame(data_for_df)
        df.to_csv("./mult_winners_data.csv", index=False)

        df2 = pd.DataFrame(data_for_df2)
        df2.to_csv("./mult_winners_data_size_ws.csv", index=False)


## Percent of Different Winners from Split Cycle

In [ ]:
def record_sc_diff_data(vms, num_cands, num_voters, pm, t): 
    if not hasattr(record_sc_diff_data, 'seed_set'):
        init_process()
        record_sc_diff_data.seed_set = True
    prof = generate_profile(num_cands, num_voters, probmod = pm )
    
    winning_sets = {vm.name: vm(prof) for vm in vms}
    sc_ws = split_cycle(prof)
    return {vm.name: {
        "diff_sc": winning_sets[vm.name] != sc_ws if winning_sets[vm.name] is not None else None, 
        "diff_sc_ec": (winning_sets[vm.name] != sc_ws) and (np.random.choice(winning_sets[vm.name]) != np.random.choice(sc_ws)) if winning_sets[vm.name] is not None else None, 
        "refinement_sc": (all([w  in sc_ws for w in winning_sets[vm.name]]) and any([w not in winning_sets[vm.name] for w in sc_ws])) if winning_sets[vm.name] is not None else None,
        "supset_sc": (all([w in winning_sets[vm.name] for w in sc_ws]) and any([w not in sc_ws for w in winning_sets[vm.name]])) if winning_sets[vm.name] is not None else None,
        "disjoint_sc": (not any([w  in winning_sets[vm.name] for w in sc_ws]) and not any([w  in sc_ws for w in winning_sets[vm.name]])) if winning_sets[vm.name] is not None else None
    } 
            for vm in vms
           }


In [ ]:


vms = [
    beat_path_Floyd_Warshall, 
]

num_trials = 25_000

all_num_voters = [
    (4,5),
    (10, 11), 
    (20, 21), 
    (50, 51), 
    (100, 101),
    (500, 501),
    (1000, 1001),
    (5000, 5001)
]

all_num_cands = [
    5,
    7, 
    10, 
    20, 
    30
]

prob_models = [
    # "IC", 
    # "IAC",
    # "URN-R",
    # "MALLOWS-RELPHI-R",
    "MALLOWS_2REF-RELPHI-R"
]

cpus = cpu_count()
print(f'CPUS: {cpus}')
num_cpus = 10

pool = Pool(num_cpus)

In [ ]:
%%time 

data_for_df = {
    "num_cands": list(), 
    "num_voters": list(),
    "pm": list(),
    "vm": list(),
    "perc_diff_sc": list(),
    "perc_diff_sc_ec": list(),
    "perc_refine_sc": list(),
    "perc_supset_sc": list(),
    "perc_disjoint_sc": list(),
    "num_profiles": list(),
}

for pm in prob_models:
    print(pm, "\n")
    for num_cands in all_num_cands:
        print(num_cands)
        for  num_voters_even, num_voters_odd in all_num_voters:
            
            print(f"\t {num_voters_even}, {num_voters_odd}")

            get_data_even = partial(record_sc_diff_data, 
                                    vms,
                                     num_cands,
                                     num_voters_even,
                                     pm
                                    )

            get_data_odd = partial(record_sc_diff_data, 
                                   vms,
                                   num_cands,
                                   num_voters_odd,
                                   pm
                                  )

            data_even = pool.map(get_data_even, range(num_trials))
            data_odd = pool.map(get_data_odd, range(num_trials))

            data =  data_even + data_odd

            for vm in vms: 
                data_for_df["num_cands"].append(num_cands)
                data_for_df["num_voters"].append(f"({num_voters_even}, {num_voters_odd})")
                data_for_df["vm"].append(vm.name)
                data_for_df["pm"].append(pm)
                _data = [d for d in data if d[vm.name]["diff_sc"] is not None]
                num_profiles = len(_data) 
                data_for_df["num_profiles"].append(num_profiles)
                if num_profiles > 0:                
                    data_for_df["perc_diff_sc"].append(len([1 for d in _data if  d[vm.name]["diff_sc"]])/ num_profiles)
                    data_for_df["perc_diff_sc_ec"].append(len([1 for d in _data if  d[vm.name]["diff_sc_ec"]])/ num_profiles)
                else: 
                    data_for_df["perc_diff_sc"].append(0)
                
                if num_profiles > 0:                
                    data_for_df["perc_refine_sc"].append(len([1 for d in _data if   d[vm.name]["refinement_sc"]])/ num_profiles) 
                else: 
                    data_for_df["perc_refine_sc"].append(0)
                
                if num_profiles > 0:                
                    data_for_df["perc_supset_sc"].append(len([1 for d in _data if d[vm.name]["supset_sc"]])/ num_profiles) 
                else: 
                    data_for_df["perc_supset_sc"].append(0)
 
                if num_profiles > 0:                
                    data_for_df["perc_disjoint_sc"].append(len([1 for d in _data if d[vm.name]["disjoint_sc"]])/ num_profiles) 
                else: 
                    data_for_df["perc_disjoint_sc"].append(0)
                               
        df = pd.DataFrame(data_for_df)
        df.to_csv("./diff_sc_data.csv", index=False)

